# **Standardizing Images**

## **Libraries**

In [1]:
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets
import torch.nn.functional as F
import numpy as np
import torch
import time

In [2]:
if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True

## **Settings**

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
random_seed = 1
learning_rate = 0.05
num_epochs = 10
batch_size = 128

In [5]:
num_classes = 10

Compute the Mean and Standard Deviation for Normalization

In [6]:
train_dataset = datasets.MNIST(root='data', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=False)

In [7]:
train_mean = []
train_std = []

for i, image in enumerate(train_loader, 0):
    numpy_image = image[0].numpy()
    
    batch_mean = np.mean(numpy_image, axis=(0, 2, 3))
    batch_std = np.std(numpy_image, axis=(0, 2, 3))
    
    train_mean.append(batch_mean)
    train_std.append(batch_std)

In [8]:
train_mean = torch.tensor(np.mean(train_mean, axis=0))
train_std = torch.tensor(np.mean(train_std, axis=0))

In [9]:
print('Mean:', train_mean)
print('Std Dev:', train_std)

Mean: tensor([0.1307])
Std Dev: tensor([0.3077])


Standardized Dataset Loader

In [10]:
custom_transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=train_mean, std=train_std)])

## **MNIST dataset**

In [11]:
train_dataset = datasets.MNIST(root='data', train=True, transform=custom_transform, download=True)
test_dataset = datasets.MNIST(root='data', train=False, transform=custom_transform)

In [12]:
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [13]:
for images, labels in train_loader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', labels.shape)
    break

Image batch dimensions: torch.Size([128, 1, 28, 28])
Image label dimensions: torch.Size([128])


In [14]:
print('Channel mean:', torch.mean(images[:, 0, :, :]))
print('Channel std:', torch.std(images[:, 0, :, :]))

Channel mean: tensor(0.0120)
Channel std: tensor(1.0132)


## **Building the model**

In [15]:
class ConvNet(torch.nn.Module):

    def __init__(self, num_classes):
        super(ConvNet, self).__init__()

        # 28x28x1 => 28x28x4
        self.conv_1 = torch.nn.Conv2d(in_channels=1,
                                      out_channels=4,
                                      kernel_size=(3, 3),
                                      stride=(1, 1),
                                      padding=1) # (1(28-1) - 28 + 3) / 2 = 1
        # 28x28x4 => 14x14x4
        self.pool_1 = torch.nn.MaxPool2d(kernel_size=(2, 2),
                                         stride=(2, 2),
                                         padding=0) # (2(14-1) - 28 + 2) = 0                                       
        # 14x14x4 => 14x14x8
        self.conv_2 = torch.nn.Conv2d(in_channels=4,
                                      out_channels=8,
                                      kernel_size=(3, 3),
                                      stride=(1, 1),
                                      padding=1) # (1(14-1) - 14 + 3) / 2 = 1                 
        # 14x14x8 => 7x7x8                             
        self.pool_2 = torch.nn.MaxPool2d(kernel_size=(2, 2),
                                         stride=(2, 2),
                                         padding=0) # (2(7-1) - 14 + 2) = 0
        
        self.linear_1 = torch.nn.Linear(7*7*8, num_classes)

        
    def forward(self, x):
        out = self.conv_1(x)
        out = F.relu(out)
        out = self.pool_1(out)

        out = self.conv_2(out)
        out = F.relu(out)
        out = self.pool_2(out)
        
        logits = self.linear_1(out.view(-1, 7*7*8))
        probas = F.softmax(logits, dim=1)
        return logits, probas

In [16]:
torch.manual_seed(random_seed)
model = ConvNet(num_classes=num_classes)

In [17]:
model = model.to(device)

In [18]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)  

## **Training phase**

In [19]:
def compute_accuracy(model, data_loader):
    correct_pred, num_examples = 0, 0
    for features, targets in data_loader:
        features = features.to(device)
        targets = targets.to(device)
        logits, probas = model(features)
        _, predicted_labels = torch.max(probas, 1)
        num_examples += targets.size(0)
        correct_pred += (predicted_labels == targets).sum()
    return correct_pred.float()/num_examples * 100

In [21]:
start_time = time.time()
for epoch in range(num_epochs):
    model = model.train()
    for batch_idx, (features, targets) in enumerate(train_loader):
        
        features = features.to(device)
        targets = targets.to(device)

        logits, probas = model(features)
        cost = F.cross_entropy(logits, targets)
        optimizer.zero_grad()
        
        cost.backward()
        optimizer.step()
        if not batch_idx % 50:
            print ('Epoch: %03d/%03d | Batch %03d/%03d | Cost: %.4f' 
                   %(epoch+1, num_epochs, batch_idx, 
                     len(train_loader), cost))
    
    model = model.eval()
    print('Epoch: %03d/%03d training accuracy: %.2f%%' % (
          epoch+1, num_epochs, 
          compute_accuracy(model, train_loader)))
    
    print('Time elapsed: %.2f min' % ((time.time() - start_time)/60))

Epoch: 001/010 | Batch 000/469 | Cost: 2.3226
Epoch: 001/010 | Batch 050/469 | Cost: 0.6869
Epoch: 001/010 | Batch 100/469 | Cost: 0.3672
Epoch: 001/010 | Batch 150/469 | Cost: 0.1709
Epoch: 001/010 | Batch 200/469 | Cost: 0.1696
Epoch: 001/010 | Batch 250/469 | Cost: 0.1216
Epoch: 001/010 | Batch 300/469 | Cost: 0.1375
Epoch: 001/010 | Batch 350/469 | Cost: 0.1643
Epoch: 001/010 | Batch 400/469 | Cost: 0.1692
Epoch: 001/010 | Batch 450/469 | Cost: 0.1302
Epoch: 001/010 training accuracy: 93.55%
Time elapsed: 0.94 min
Epoch: 002/010 | Batch 000/469 | Cost: 0.3376
Epoch: 002/010 | Batch 050/469 | Cost: 0.0870
Epoch: 002/010 | Batch 100/469 | Cost: 0.1786
Epoch: 002/010 | Batch 150/469 | Cost: 0.1353
Epoch: 002/010 | Batch 200/469 | Cost: 0.1302
Epoch: 002/010 | Batch 250/469 | Cost: 0.1282
Epoch: 002/010 | Batch 300/469 | Cost: 0.0811
Epoch: 002/010 | Batch 350/469 | Cost: 0.1510
Epoch: 002/010 | Batch 400/469 | Cost: 0.1340
Epoch: 002/010 | Batch 450/469 | Cost: 0.0909
Epoch: 002/010 t

In [22]:
print('Total Training Time: %.2f min' % ((time.time() - start_time)/60))

Total Training Time: 10.85 min


## **Evaluation**

In [23]:
print('Test accuracy: %.2f%%' % (compute_accuracy(model, test_loader)))

Test accuracy: 98.28%
